# Colab flow

### GPU usage

To use GPU in colab: `Edit > Notebook settings or Runtime>Change runtime type and select GPU as Hardware accelerator`.

### Setting repo

In [1]:
colab = False

For `colab=True` you need to rewrite the `branch` and `github_token` below.

In [2]:
if colab:
    !pip install condacolab # Handling conda environment installation
    import condacolab
    condacolab.install()
    !git clone -b branch https://github_token@github.com/Nixtla/nixtlats.git
    %cd nixtlats
    !mamba env update -n base -f environment.yml
    !pip install -e .

# Standard flow

In [1]:
import numpy as np
import pytorch_lightning as pl
import torch as t
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning import seed_everything

from nixtla.data.datasets.m4 import M4, M4Info, M4Evaluation
from nixtla.data.tsdataset import TimeSeriesDataset
from nixtla.data.tsloader import TimeSeriesLoader
from nixtla.models.esrnn.esrnn import ESRNN

In [2]:
seed_everything(117982, workers=True)

Global seed set to 117982


117982

## Import Monthly data

In [3]:
group = M4Info['Monthly']
Y_df, _, S_df = M4.load(directory='data', group=group.name)

In [4]:
train_ts_dataset = TimeSeriesDataset(Y_df=Y_df, 
                                     S_df=S_df,
                                     ds_in_test=group.horizon,
                                     mode='full',
                                     window_sampling_limit=272, # To limit backprop time
                                     input_size=12,
                                     output_size=group.horizon,
                                     idx_to_sample_freq=1,
                                     len_sample_chunks=60,
                                     complete_inputs=True, 
                                     skip_nonsamplable=True)

In [5]:
test_ts_dataset = TimeSeriesDataset(Y_df=Y_df, 
                                    S_df=S_df,
                                    ds_in_test=0,
                                    mode='full',
                                    window_sampling_limit=200_000, # To limit backprop time
                                    input_size=12,
                                    output_size=group.horizon,
                                    idx_to_sample_freq=1,
                                    complete_inputs=True,
                                    complete_outputs=True,
                                    len_sample_chunks=60,
                                    skip_nonsamplable=False,
                                    last_samplable_window=True)

In [6]:
train_ts_loader = TimeSeriesLoader(dataset=train_ts_dataset,
                                   batch_size=32,
                                   eq_batch_size=True,
                                   num_workers=8,
                                   shuffle=True)

In [7]:
test_ts_loader = TimeSeriesLoader(dataset=test_ts_dataset,
                                  batch_size=2048,
                                  eq_batch_size=False,
                                  num_workers=8,
                                  shuffle=False)

In [8]:
model = ESRNN(n_series=group.n_ts,
              n_x=0, n_s=1,
              idx_to_sample_freq=1,
              input_size=12,
              output_size=group.horizon,
              learning_rate=0.0128,
              lr_scheduler_step_size=15,
              lr_decay=0.1744,
              per_series_lr_multip=1,
              gradient_clipping_threshold=20,
              rnn_weight_decay=0,
              level_variability_penalty=55,
              testing_percentile=50,
              training_percentile=45,
              cell_type='LSTM',
              state_hsize=55,
              dilations=[[1, 3, 6, 12]],
              add_nl_layer=False,
              loss='SMYL',
              val_loss='SMAPE',
              seasonality=[12])

In [9]:
trainer = pl.Trainer(max_epochs=20, progress_bar_refresh_rate=5, deterministic=True)
trainer.fit(model, train_ts_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
/home/ubuntu/anaconda3/envs/nixtla/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: you defined a validation_step but have no val_dataloader. Skipping val loop
  warnings.warn(*args, **kwargs)

  | Name  | Type   | Params
---------------------------------
0 | esrnn | _ESRNN | 762 K 
---------------------------------
762 K     Trainable params
0         Non-trainable params
762 K     Total params
3.049     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 117982


Training: 0it [00:00, ?it/s]

In [10]:
outputs = trainer.predict(model, test_ts_loader)

Predicting: 1500it [00:00, ?it/s]

In [11]:
_, y_hat = zip(*outputs)

In [12]:
y_hat = t.cat(y_hat)[:, -1].cpu().numpy()

*Note*: this is an individual model rather than the ensemble used by Smyl which gets an OWA of *0.836*.

In [13]:
M4Evaluation.evaluate('data', 'Monthly', y_hat)

,SMAPE,MASE,OWA
Monthly,12.529585,0.8956,0.855364
